In [391]:
import numpy as np
from numpy import kron, eye as I, diag
import matplotlib.pyplot as plt

from ndgsp.graph.graphs import Graph

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

import sys
sys.path.append('..')

from utils.plotting import to_colors


In [307]:
np.random.seed(0)


N = 50
M = 5
lam = 0.4
gamma = 0.5

g = Graph.random_connected(N)


U = g.U

# DG = diag(np.exp(-g.lam))
# DG = I(N)
DG = diag([1] + [0] * (N - 1))

m = 0.8
DS = np.zeros(N)
DS[np.random.choice(np.arange(N), size= int((1 - m) * N))] = 1
DS = diag(DS)

X = np.random.normal(size=(N, M))

lamM, UM = np.linalg.eigh(X.T @ DS @ X)
DM = diag((lamM + lam) ** -0.5)

M11 = DG @ U.T @ DS @ U @ DG + gamma * I(N)
M12 = DG @ U.T @ DS @ X @ UM @ DM

M = np.block([[M11, M12], [M12.T, I(M)]])

lams = np.linalg.eigvalsh(M)

In [311]:
Graph.random_connected(8)

Graph(N=8)

In [422]:
def make_kgr_tikz_graph():
    """
    Create the TikZ code for plotting the KGR diagram
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({-1.5 + offset},2) node{{$t = {t}$}}; \n"
        tex += f"\\draw ({-3.5 + offset},0) node{{ $ \\mathbf{{x}}_{t} = \\begin{{bmatrix}} {x[0]:.2f} \\\\[0.1cm] {x[1]:.2f} \\\\[0.1cm] {x[2]:.2f} \\\\[0.1cm] {x[3]:.2f} \\end{{bmatrix}}$ }};\n"
        tex +=  f"\\draw ({-1.6 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=1mm] ({i + j * 10}) at ({offset + xx[i] / 6:.2f}, {yy[i] / 6:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)

    all_tex += '\\draw (1.3, 0) node{$,$}; \n \\draw (8.9, 0) node{$, \\quad\\quad \\dots $};\n\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    return all_tex 

    

In [423]:
print(make_kgr_tikz_graph())

\documentclass[crop,tikz]{standalone} 
\usepackage{tikz, amsmath, amssymb} 
\usetikzlibrary{positioning, shapes.geometric} 
\begin{document} 
\begin{tikzpicture}
\draw (-1.5,2) node{$t = 1$}; 
\draw (-3.5,0) node{ $ \mathbf{x}_1 = \begin{bmatrix} -0.47 \\[0.1cm] 0.97 \\[0.1cm] -1.28 \\[0.1cm] 1.44 \end{bmatrix}$ };
\draw (-1.6,0) node{$\mathbf{y}_1 = $};

\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=1mm] (0) at (0.34, -0.00) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,0}, minimum size=1mm] (1) at (1.35, 0.88) {};
\node[circle, draw, fill={rgb,255: red,255; green,18; blue,0}, minimum size=1mm] (2) at (-1.05, -0.72) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=1mm] (3) at (-0.04, 1.53) {};
\node[circle, draw, fill={rgb,255: red,255; green,0; blue,0}, minimum size=1mm] (4) at (-0.56, -1.31) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=1mm] (5) at (-0.74, 0.84) 

In [443]:
def make_rnc_tikz_graph():
    """
    Create the TikZ code for plotting the RNC diagram
    """

    np.random.seed(1)

    x = 2 * np.random.uniform(0, 10, size=10)
    y = 2 * np.random.uniform(0, 10, size=10)

    xx = x - x.mean()
    yy = y - y.mean()

    edges  = np.array([(0, 9),(0, 8),(0, 7),(0, 6),(0, 1),(1, 9),(1, 7),(1, 3),(3, 7),(3, 5), 
                       (6, 7),(5, 6),(6, 8),(2, 5),(2, 6),(2, 4),(4, 6),(4, 8),(8, 9), (7, 5)])


    A = np.zeros((10, 10))

    A[edges[:, 0], edges[:, 1]] = 1
    A = A + A.T

    L = np.diag(A.sum(0)) - A 

    lam, U = np.linalg.eigh(L)

    H = U @ diag(np.exp(-lam)) @ U.T

    def make_section(i, t, offset):

        np.random.seed(i)

        s = np.random.choice(np.arange(10), size=4)

        cols = to_colors(H @ np.random.normal(size=10), cmap='autumn')

        x = np.random.normal(size=4)

        tex = f"\\draw ({0 + offset},3) node{{$t = {t}$}}; \n"
        tex +=  f"\\draw ({-2.3 + offset},0) node{{$\\mathbf{{y}}_{t} = $}};\n\n"

        for i in range(10):

            if i in s:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,255; green,255; blue,255}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

            else:
                tex += f'\\node[circle, draw, fill={{rgb,255: red,{int(cols[i][1:3], base=16)}; green,{int(cols[i][3:5], base=16)}; blue,{int(cols[i][5:], base=16)}}}, minimum size=2mm] ({i + j * 10}) at ({offset + xx[i] / 4:.2f}, {yy[i] / 4:.2f}) {{}};\n'

        tex += '\n'

        for e1, e2 in edges:
            tex += f'\draw[gray, very thin] ({e1 + j * 10}) to ({e2 + j * 10});\n'

        for xi, yi in [(0.8, 2.3), (-2, 1.2), (0.2, 1.1), (-0.7, -0.4), (1.45, 0), (2, 1.7), (-0.8, -2.3), (-1.6, -1.5), (0.5, -1.8), (2, -1.1)]:

            x_ = np.abs(np.random.normal(size=3))
            tex += f'\\draw ({xi + offset}, {yi}) node{{\\scalebox{{0.7}}{{$[{x_[0]:.1f}, {x_[1]:.1f}, {x_[2]:.1f}]$}}}};\n'
 
        return tex + '\n'

    all_tex = """\\documentclass[crop,tikz]{standalone} \n\\usepackage{tikz, amsmath, amssymb} \n\\usetikzlibrary{positioning, shapes.geometric} \n\\begin{document} \n\\begin{tikzpicture}\n"""

    for j, t in enumerate(['1', '2', 'T']):

        if j == 0:
            o = 0
        elif j == 1:
            o = 7
        else:
            o = 15

        all_tex += make_section(j, t, o)


    all_tex += "\draw (2.3, 0) node{$,$};\n" 
    all_tex += '\draw (10.2, 0) node{$, \quad\quad\quad \dots $};\n'

    all_tex += "\\end{tikzpicture}\n\\end{document} "

    return all_tex 

    

In [444]:
print(make_rnc_tikz_graph())

\documentclass[crop,tikz]{standalone} 
\usepackage{tikz, amsmath, amssymb} 
\usetikzlibrary{positioning, shapes.geometric} 
\begin{document} 
\begin{tikzpicture}
\draw (0,3) node{$t = 1$}; 
\draw (-2.3,0) node{$\mathbf{y}_1 = $};

\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=2mm] (0) at (0.51, -0.00) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,0}, minimum size=2mm] (1) at (2.03, 1.33) {};
\node[circle, draw, fill={rgb,255: red,255; green,18; blue,0}, minimum size=2mm] (2) at (-1.57, -1.08) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=2mm] (3) at (-0.06, 2.29) {};
\node[circle, draw, fill={rgb,255: red,255; green,0; blue,0}, minimum size=2mm] (4) at (-0.84, -1.96) {};
\node[circle, draw, fill={rgb,255: red,255; green,255; blue,255}, minimum size=2mm] (5) at (-1.11, 1.25) {};
\node[circle, draw, fill={rgb,255: red,255; green,97; blue,0}, minimum size=2mm] (6) at (-0.64, -0.01) {};
\node[circle, dr